# Resume Chatbot

## Web Analytics Final project

This is part of the final project on 620 Web Analytics course. 

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [2]:
from keras.layers import GRU, Input, Dense, TimeDistributed, SimpleRNN, LSTM
from keras.models import Model
from keras.models import Sequential
from keras.layers import Activation
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.callbacks import ModelCheckpoint
from keras.layers import Dropout
from keras.layers import Dropout, Bidirectional, RepeatVector
from keras.layers.embeddings import Embedding
import collections
import pandas as pd
import nltk

from keras.preprocessing.text import Tokenizer,text_to_word_sequence
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from autocorrect import spell
from scipy.spatial import distance
import time

In [3]:
df = pd.read_csv('./data/final_qa_data.csv',sep=',')
df.head()

,Question,Answer
0,Tell me 5 positive things about you,Tricky question detecting. Waiting for next qu...
1,Tell me your strengths,Tricky question detecting. Waiting for next qu...
2,Tell us Unique Selling Points,Tricky question detecting. Waiting for next qu...
3,What are you good at ?,Tricky question detecting. Waiting for next qu...
4,What are your professional strengths ?,Tricky question detecting. Waiting for next qu...


In [4]:
question_sentences = df.Question.values
answer_sentences = df.Answer.values

print('Dataset Loaded')

Dataset Loaded


In [5]:
len(answer_sentences)

261

### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [6]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, question_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, answer_sentences[sample_i]))

small_vocab_en Line 1:  Tell me 5 positive things about you
small_vocab_fr Line 1:  Tricky question detecting. Waiting for next question...
small_vocab_en Line 2:  Tell me your strengths
small_vocab_fr Line 2:  Tricky question detecting. Waiting for next question...


In [7]:
text_to_word_sequence(question_sentences[0])

['tell', 'me', '5', 'positive', 'things', 'about', 'you']

From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [8]:
def preprocess_cleaning(texts):
    
    text_cleaned = []
    
#    for text in texts:
#        for raw_word in text_to_word_sequence(text,filters='', lower=True):
            #
#           l1 = ['won’t','won\'t','wouldn’t','wouldn\'t','’m', '’re', '’ve', '’ll', '’s','’d', 'n’t', '\'m', '\'re', '\'ve', '\'ll', '\'s', '\'d', 'can\'t', 'n\'t', 'B: ', 'A: ', ',', ';', '.', '?', '!', ':', '. ?', ',   .', '. ,', 'EOS', 'BOS', 'eos', 'bos']
#            l2 = ['will not','will not','would not','would not',' am', ' are', ' have', ' will', ' is', ' had', ' not', ' am', ' are', ' have', ' will', ' is', ' had', 'can not', ' not', '', '', ' ,', ' ;', ' .', ' ?', ' !', ' :', '? ', '.', ',', '', '', '', '']

#            raw_word = raw_word.lower()

#            for j, term in enumerate(l1):
#                raw_word = raw_word.replace(term,l2[j])

#            for j in range(30):
#                raw_word = raw_word.replace('. .', '')
#                raw_word = raw_word.replace('.  .', '')
#                raw_word = raw_word.replace('..', '')
#                raw_word = raw_word.replace('...', '')

#            for j in range(5):
#                raw_word = raw_word.replace('  ', ' ')
                
#            text_cleaned.append(raw_word)
            
        #text_cleaned.append(raw_word)
        #print(text_cleaned)
        #Convert to lower and tokenize
        
        #text_cleaned.append(' '.join(text_to_word_sequence(raw_word,filters='', lower=True)))
        
    text_cleaned = [' '.join(nltk.tokenize.word_tokenize(word.lower(),preserve_line=True)) for word in texts]
        
        
        
    return text_cleaned
    #return [' '.join(text_to_word_sequence(text,filters='', lower=True)) for text in raw_word]
    

In [9]:
question_sentences = preprocess_cleaning(question_sentences)
answer_sentences = preprocess_cleaning(answer_sentences)

#[' '.join(text_to_word_sequence(text)) for text in question_sentences]

In [10]:

question_words_counter = collections.Counter([word for sentence in question_sentences for word in sentence.split()])
answer_words_counter = collections.Counter([word for sentence in answer_sentences for word in sentence.split()])

print('{} words in questions.'.format(len([word for sentence in question_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(question_words_counter)))
print('10 Most common words in the questions dataset:')
print('"' + '" "'.join(list(zip(*question_words_counter.most_common(10)))[0]) + '"')
print()
print('{} words in answers.'.format(len([word for sentence in answer_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(answer_words_counter)))
print('10 Most common words in the answers dataset:')
print('"' + '" "'.join(list(zip(*answer_words_counter.most_common(10)))[0]) + '"')

1124 words in questions.
288 unique English words.
10 Most common words in the questions dataset:
"you" "?" "what" "do" "are" "your" "how" "i" "'s" "tell"

3361 words in answers.
292 unique English words.
10 Most common words in the answers dataset:
"i" "you" ":" "to" "," "my" "..." "a" "and" "."


In [11]:
question_words_counter.most_common(20)

[('you', 110),
 ('?', 102),
 ('what', 66),
 ('do', 38),
 ('are', 36),
 ('your', 31),
 ('how', 27),
 ('i', 19),
 ("'s", 19),
 ('tell', 17),
 ('me', 17),
 ('about', 17),
 ('to', 16),
 ('a', 15),
 ('is', 15),
 ('can', 13),
 ('work', 13),
 ('the', 13),
 ('have', 12),
 ('where', 10)]

For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `question_sentences` and `answer_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [12]:


def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    x = [' '.join(nltk.tokenize.word_tokenize(word.lower(),preserve_line=True)) for word in x]

    tokenizer = Tokenizer(num_words=None, filters="", lower=True, split=" ")
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer


# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog.',
    'By Jove , my quick study of lexicography won a prize.',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'.': 1, 'the': 2, 'quick': 3, 'a': 4, 'brown': 5, 'fox': 6, 'jumps': 7, 'over': 8, 'lazy': 9, 'dog': 10, 'by': 11, 'jove': 12, ',': 13, 'my': 14, 'study': 15, 'of': 16, 'lexicography': 17, 'won': 18, 'prize': 19, 'this': 20, 'is': 21, 'short': 22, 'sentence': 23}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog.
  Output: [2, 3, 5, 6, 7, 8, 2, 9, 10, 1]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize.
  Output: [11, 12, 13, 14, 3, 15, 16, 17, 18, 4, 19, 1]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [20, 21, 4, 22, 23, 1]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [13]:


def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    if length==None:
        maxLenX = 0
        for sequence in x:
            if len(sequence) > maxLenX:
                maxLenX = len(sequence)
        
        padded = pad_sequences(sequences=x,maxlen=maxLenX, dtype='int32', padding='post', truncating='post', value=0)

    else:
        padded = pad_sequences(sequences=x,maxlen=length, dtype='int32', padding='post', truncating='post', value=0)
    return padded


# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [ 2  3  5  6  7  8  2  9 10  1]
  Output: [ 2  3  5  6  7  8  2  9 10  1  0  0]
Sequence 2 in x
  Input:  [11 12 13 14  3 15 16 17 18  4 19  1]
  Output: [11 12 13 14  3 15 16 17 18  4 19  1]
Sequence 3 in x
  Input:  [20 21  4 22 23  1]
  Output: [20 21  4 22 23  1  0  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [14]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
    
    print(preprocess_y.shape)
    
    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_question_sentences, preproc_answer_sentences, question_tokenizer, answer_tokenizer =\
    preprocess(question_sentences, answer_sentences)

print('Data Preprocessed')


(261, 39)
Data Preprocessed


In [15]:
# Embedding matrix

#word embeddings from glove

embeddings_index = dict()

with open('./data/glove.6B.100d.txt',encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word]= coefs

print('loaded %s word vectors' % len(embeddings_index))

loaded 400000 word vectors


In [16]:
#encoder matrix
num_question_tokens = len(question_tokenizer.word_index)+1

encoder_embedding_matrix = np.zeros((num_question_tokens, 100))

unknown_helper_list = []

for word,i in question_tokenizer.word_index.items():
    #word_clean = text_to_word_sequence(word, filters='')[0]
    embedding_vector = embeddings_index.get(word)
    unknown_helper_list.append(word)

    if embedding_vector is not None:
        encoder_embedding_matrix[i] = embedding_vector

In [17]:
learning_rate = 0.01
hidden_dim = 256
embedding_vector_length = 100

In [18]:

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_question_sentences, preproc_answer_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_answer_sentences.shape[-2]))


In [19]:
input_shape =tmp_x.shape
output_sequence_length =preproc_answer_sentences.shape[1]
question_vocab_size =len(question_tokenizer.word_index)+1
answer_vocab_size=len(answer_tokenizer.word_index)+1


In [23]:
    #Variation - 3 Hidden LSTM Layer and 2 Dense ouput layer with higher Dropout Rate

model = Sequential()
model.add(Embedding(question_vocab_size, embedding_vector_length,weights=[encoder_embedding_matrix], trainable= False,
                    input_shape= input_shape[1:]))
model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Dense(answer_vocab_size))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(answer_vocab_size))
model.add(Activation('softmax'))
model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(learning_rate),
              metrics=['accuracy'])
print(model.summary())

epochs =1000

model.fit(tmp_x, preproc_answer_sentences, batch_size=128, epochs=epochs, validation_split=0.2)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 39, 100)           28900     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 39, 512)           731136    
_________________________________________________________________
dropout_1 (Dropout)          (None, 39, 512)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 39, 512)           1574912   
_________________________________________________________________
dropout_2 (Dropout)          (None, 39, 512)           0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 39, 512)           1574912   
_________________________________________________________________
dropout_3 (Dropout)          (None, 39, 512)           0         
__________

In [35]:
model.save('./saved_models/resume_chatbot_save_'+str(epochs)+'.hdf5')

## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [20]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [21]:
def similar_word(unknown_word_dim):

    all_distance = []

    for known_word in encoder_embedding_matrix:
        all_distance.append(distance.euclidean(unknown_word_dim,known_word))
    
    return(unknown_helper_list[np.array(all_distance).argsort()[:1][0]])

In [22]:
def preprocess_test(raw_word, question_tokenizer):
    l1 = ['won’t','won\'t','wouldn’t','wouldn\'t','’m', '’re', '’ve', '’ll', '’s','’d', 'n’t', '\'m', '\'re', '\'ve', '\'ll', '\'s', '\'d', 'can\'t', 'n\'t', 'B: ', 'A: ', ',', ';', '.', '?', '!', ':', '. ?', ',   .', '. ,', 'EOS', 'BOS', 'eos', 'bos']
    l2 = ['will not','will not','would not','would not',' am', ' are', ' have', ' will', ' is', ' had', ' not', ' am', ' are', ' have', ' will', ' is', ' had', 'can not', ' not', '', '', ' ,', ' ;', ' .', ' ?', ' !', ' :', '? ', '.', ',', '', '', '', '']

    raw_word = raw_word.lower()

    for j, term in enumerate(l1):
        raw_word = raw_word.replace(term,l2[j])
       
    for j in range(30):
        raw_word = raw_word.replace('. .', '')
        raw_word = raw_word.replace('.  .', '')
        raw_word = raw_word.replace('..', '')
        raw_word = raw_word.replace('...', '')
        
    for j in range(5):
        raw_word = raw_word.replace('  ', ' ')
 
    #Spell checker
    
    #text_cleaned = [' '.join(nltk.tokenize.word_tokenize(word.lower(),preserve_line=True)) for word in texts]
    #print(raw_word)

    #raw_word = ' '.join([spell(text) for text in nltk.tokenize.word_tokenize(raw_word.lower(),preserve_line=True)])
    #print(raw_word)
    
    final_corrected_words  = []
    
    for text in nltk.tokenize.word_tokenize(raw_word.lower(),preserve_line=True):
        text = text.lower()
        #print(text)
        if text not in '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n':
            text = spell(text).lower()
            if text not in question_tokenizer.word_index.keys():
                print(text)
                try:
                    final_corrected_words.append(similar_word(embeddings_index[text]))
                except:
                    final_corrected_words.append('?')
            else:
                final_corrected_words.append(text)
        else:
           
            if text not in question_tokenizer.word_index.keys():
                final_corrected_words.append(similar_word(embeddings_index[text]))
            else:
                final_corrected_words.append(text)
            
    #print(' '.join(final_corrected_words))
    return ' '.join(final_corrected_words)


In [23]:


que = ''
last_query  = ' '
last_last_query = ''
text = ' '
last_text = ''
name_of_computer = 'Jammy'

# Open files to save the conversation for further training:

name = time.strftime("%Y%m%d-%H%M%S")
qf = open('./session_data/'+name+'.txt', 'w')
#af = open(file_saved_answer, 'w')

def final_predictions(model, x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    
    #model = load_model('saved_models/final_rnn_model6.hdf5')
    #model.summary()
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'
    

    
    print('computer: hi ! please type your name.\n')
    name = input('user: ')
    print('computer: hi , ' + name +' ! My name is ' + name_of_computer + '.\n') 

    
    while(True):
        
        que = input()#'saw a old yellow'
        
        qf.write("Question typed:" + que + '\n')
        
        if que =='exit':
            break
        else:
            que = preprocess_test(que,x_tk)
            #print(que)
        sentence =que
        
        qf.write("Question interpreted:" + que + '\n')
        
        sentence = [x_tk.word_index[word] for word in text_to_word_sequence(sentence,filters='')]
        

        sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
        sentences = np.array([sentence[0], x[0]])

        tmp_sentences = pad(sentences, y.shape[1])
        tmp_sentences = tmp_sentences.reshape((-1, y.shape[-2]))

        predictions = model.predict(tmp_sentences, len(tmp_sentences))

        #print('Sample 1:')
        prediction_text = ' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]).replace('<PAD>','')
        
        qf.write("Answer by bot:" + prediction_text + '\n')
        print(prediction_text)
        #print('Sample 2:')
        #print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
        #print(' '.join([y_id_to_word[np.argmax(x)] for x in y[0]]))
    
    qf.close()

final_predictions(loaded_model,preproc_question_sentences, preproc_answer_sentences, question_tokenizer, answer_tokenizer)

computer: hi ! please type your name.

user: Shyam
computer: hi , Shyam ! My name is Jammy.

Hi
greetings !                                     
Hi
greetings !                                     
how are you?
i 'm perfectly fine. the servers are paid and in case of network failure i 'll automatically be available from a different region .               
what salary?
i wonder where one electricity chronicles now. ...  your thoughts this ?                          
your portfolio?
my open source you from time see on github : are https : //github.com/bvshyam/. this bot also an achievement :                   
joke?
ok.. here it is. roses are \ # ff0000 / violets are \ # 0000ff / all my base / belong to you .               
your positive?
my open source projects. you can see them on github : https : //github.com/bvshyam/. this bot is also an achievement : )                 
your weakness?
tricky question detecting. waiting for next ...                                
all right
good it you

In [30]:
qf.close()

In [1]:
from keras.models import load_model

#loaded_model = load_model('saved_models/final_rnn_model6.hdf5')
loaded_model = load_model('saved_models/resume_chatbot_save_2000.hdf5')

loaded_model.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 39, 100)           28900     
_________________________________________________________________
bidirectional_4 (Bidirection (None, 39, 512)           731136    
_________________________________________________________________
dropout_5 (Dropout)          (None, 39, 512)           0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 39, 512)           1574912   
_________________________________________________________________
dropout_6 (Dropout)          (None, 39, 512)           0         
_________________________________________________________________
bidirectional_6 (Bidirection (None, 39, 512)           1574912   
_________________________________________________________________
dropout_7 (Dropout)          (None, 39, 512)           0         
__________

In [ ]:
#Variation - 3 Hidden LSTM Layer and 2 Dense ouput layer with higher Dropout Rate

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_question_sentences, preproc_answer_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_answer_sentences.shape[-2]))


input_shape =tmp_x.shape
output_sequence_length =preproc_answer_sentences.shape[1]
question_vocab_size =len(question_tokenizer.word_index)+1
answer_vocab_size=len(answer_tokenizer.word_index)+1

model = Sequential()
model.add(Embedding(question_vocab_size, embedding_vector_length,weights=[encoder_embedding_matrix], trainable= False,
                    input_shape= input_shape[1:]))
model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Dense(answer_vocab_size))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(answer_vocab_size))
model.add(Activation('softmax'))
model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(learning_rate),
              metrics=['accuracy'])
print(model.summary())

epochs =1000

model.fit(tmp_x, preproc_answer_sentences, batch_size=128, epochs=epochs, validation_split=0.2)



# Different try Seq2Seq


In [ ]:
# New code 
question_context = Input(shape=(max_encoder_seq_length,), dtype='int32')
encoder_Embedding = Embedding(output_dim=input_dimenstions, input_dim=num_encoder_tokens,weights=[encoder_embedding_matrix], trainable= False)#, input_length=max_encoder_seq_length)
LSTM_encoder = LSTM(hidden_dim, kernel_initializer= 'lecun_uniform', return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

answer_context = Input(shape=(max_decoder_seq_length,), dtype='int32')
LSTM_decoder = LSTM(hidden_dim, kernel_initializer= 'lecun_uniform', return_state=True)
decoder_Embedding = Embedding(output_dim=input_dimenstions, input_dim=num_decoder_tokens, weights=[encoder_embedding_matrix], trainable= False, input_length=max_decoder_seq_length)

#Shared_Embedding = Embedding(output_dim=input_dimenstions, input_dim=dictionary_size, weights=[embedding_matrix], input_length=maxlen_input, name='Shared')

word_embedding_context = encoder_Embedding(input_context)
context_embedding = LSTM_encoder(word_embedding_context)

word_embedding_answer = decoder_Embedding(input_answer)
answer_embedding = LSTM_decoder(word_embedding_answer)

merge_layer = merge([context_embedding, answer_embedding], mode='concat', concat_axis=1)
out = Dense(input_dimenstions, activation="relu")(merge_layer)
out = Dense(input_dimenstions, activation="softmax")(out)

model = Model(input=[input_context, input_answer], output = [out])
ad = Adam(lr=0.00005) 
model.compile(loss='categorical_crossentropy', optimizer=ad)

model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=1)

In [ ]:
# Inputs 
#tmp_x, preproc_answer_sentences

# Old model

In [53]:

def model_final6(input_shape, output_sequence_length, question_vocab_size, answer_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param question_vocab_size: Number of unique English words in the dataset
    :param answer_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    print(input_shape[1:])
    print(output_sequence_length)
    
    learning_rate = 0.01
    hidden_dim = 256
    embedding_vector_length = 100
    
    #Variation - 3 Hidden LSTM Layer and 2 Dense ouput layer with higher Dropout Rate
    
    model = Sequential()
    model.add(Embedding(question_vocab_size, embedding_vector_length, weights=encoder_embedding_matrix,trainable= False,
                        input_shape= input_shape[1:]))
    model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Dense(answer_vocab_size))
    model.add(Activation('sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(answer_vocab_size))
    model.add(Activation('softmax'))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    print(model.summary())
    return model


#tests.test_model_final(model_final)


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_question_sentences, preproc_answer_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_answer_sentences.shape[-2]))

# Train the neural network
final_rnn_model6 = model_final6(
    tmp_x.shape,
    preproc_answer_sentences.shape[1],
    len(question_tokenizer.word_index)+1,
    len(answer_tokenizer.word_index)+1)
final_rnn_model6.fit(tmp_x, preproc_answer_sentences, batch_size=128, epochs=500, validation_split=0.2)

final_rnn_model6.save('./saved_models/final_rnn_model6_embed.hdf5')

# Print prediction(s)
print(logits_to_text(final_rnn_model6.predict(tmp_x[:1])[0], answer_tokenizer))

print('Final Model Loaded')

(15,)
15


ValueError: You called `set_weights(weights)` on layer "embedding_6" with a  weight list of length 2707, but the layer was expecting 1 weights. Provided weights: [[ 0.          0.          0.         ...,  0.    ...

In [29]:
model.summary()

NameError: name 'model' is not defined

In [40]:
print(logits_to_text(model.predict(tmp_x[25:30])[0], answer_tokenizer))


It <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [ ]:
#preproc_answer_sentences[6]

len(answer_tokenizer.word_index)+1
#print(logits_to_text(final_rnn_model6.predict(tmp_x[20]), answer_tokenizer))


## Prediction (IMPLEMENTATION)

In [52]:
def preprocess_test(raw_word, name, question_tokenizer):
    l1 = ['won’t','won\'t','wouldn’t','wouldn\'t','’m', '’re', '’ve', '’ll', '’s','’d', 'n’t', '\'m', '\'re', '\'ve', '\'ll', '\'s', '\'d', 'can\'t', 'n\'t', 'B: ', 'A: ', ',', ';', '.', '?', '!', ':', '. ?', ',   .', '. ,', 'EOS', 'BOS', 'eos', 'bos']
    l2 = ['will not','will not','would not','would not',' am', ' are', ' have', ' will', ' is', ' had', ' not', ' am', ' are', ' have', ' will', ' is', ' had', 'can not', ' not', '', '', ' ,', ' ;', ' .', ' ?', ' !', ' :', '? ', '.', ',', '', '', '', '']
#   l3 = ['-', '_', ' *', ' /', '* ', '/ ', '\"', ' \\"', '\\ ', '--', '...', '. . .']
#    l4 = ['jeffrey','fred','benjamin','paula','walter','rachel','andy','helen','harrington','kathy','ronnie','carl','annie','cole','ike','milo','cole','rick','johnny','loretta','cornelius','claire','romeo','casey','johnson','rudy','stanzi','cosgrove','wolfi','kevin','paulie','cindy','paulie','enzo','mikey','i\97','davis','jeffrey','norman','johnson','dolores','tom','brian','bruce','john','laurie','stella','dignan','elaine','jack','christ','george','frank','mary','amon','david','tom','joe','paul','sam','charlie','bob','marry','walter','james','jimmy','michael','rose','jim','peter','nick','eddie','johnny','jake','ted','mike','billy','louis','ed','jerry','alex','charles','tommy','bobby','betty','sid','dave','jeffrey','jeff','marty','richard','otis','gale','fred','bill','jones','smith','mickey']    

    raw_word = raw_word.lower()
    raw_word = raw_word.replace(', ' + name_of_computer, '')
    raw_word = raw_word.replace(name_of_computer + ' ,', '')

    for j, term in enumerate(l1):
        raw_word = raw_word.replace(term,l2[j])
        
 #   for term in l3:
 #       raw_word = raw_word.replace(term,' ')
       
    for j in range(30):
        raw_word = raw_word.replace('. .', '')
        raw_word = raw_word.replace('.  .', '')
        raw_word = raw_word.replace('..', '')
       
    for j in range(5):
        raw_word = raw_word.replace('  ', ' ')
        
    #if raw_word[-1] !=  '!' and raw_word[-1] != '?' and raw_word[-1] != '.' and raw_word[-2:] !=  '! ' and raw_word[-2:] != '? ' and raw_word[-2:] != '. ':
    #    raw_word = raw_word + ' .'
    
    #if raw_word == ' !' or raw_word == ' ?' or raw_word == ' .' or raw_word == ' ! ' or raw_word == ' ? ' or raw_word == ' . ':
    #    raw_word = 'what ?'
       
    
    #if raw_word == '  .' or raw_word == ' .' or raw_word == '  . ':
    #    raw_word = 'i do not want to talk about it .'
    
    
    #raw_new_string = []
    
    #for word in nltk.tokenize.word_tokenize(raw_word):
    #    if word not in question_tokenizer.word_index.keys():
    #        word = name
    #        raw_new_string.append(word)
    #    else:
    #        raw_new_string.append(word)
            
    
    
            #for dict_word in question_tokenizer.word_index.keys():
            #    nlp_words = nlp(word)
            #    dict_tokens = nlp(dict_word)
            #    nlp_words.similarity(dict_tokens)
    
    
    return raw_word


spacy.tokens.doc.Doc

## Submission
When you are ready to submit your project, do the following steps:
1. Ensure you pass all points on the [rubric](https://review.udacity.com/#!/rubrics/1004/view).
2. Submit the following in a zip file.
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.